In [22]:
import pandas as pd
from sklearn.decomposition import PCA
from sklearn.preprocessing import Normalizer
from sklearn.svm import SVC
import numpy as np

 - # 目的 : 用乘客搭乘的資訊來預估事發後有可能生存的人

In [23]:
data_train = pd.read_csv("titanicTrain.csv")
data_test  = pd.read_csv("titanicQuestion.csv")

In [24]:
data_train.keys()

Index(['pclass', 'survived', 'name', 'sex', 'age', 'sibsp', 'parch', 'ticket',
       'fare', 'cabin', 'embarked', 'boat', 'body', 'home.dest'],
      dtype='object')

In [25]:
data_test.keys()

Index(['pclass', 'survived', 'name', 'sex', 'age', 'sibsp', 'parch', 'ticket',
       'fare', 'cabin', 'embarked', 'boat', 'body', 'home.dest'],
      dtype='object')

 - 找有無搭上船及船編號對存貨率的影響

In [26]:
frame_boat = pd.DataFrame(data_train)

In [27]:
frame_0 = frame_boat['survived']
frame_1 = frame_boat['boat']

In [28]:
frame_s = pd.Series(frame_0[0:1000])
frame_b = pd.Series(frame_1[0:1000])
data_testt = {'survived':frame_s,'boat':frame_b}
frame_test = pd.DataFrame(data_testt)

In [29]:
frame_test.isnull().sum()

boat        583
survived      0
dtype: int64

In [30]:
frame_QQ = frame_test.sort_values(by = ['boat'] , ascending = False)
frame_QQ.index = range(len(frame_QQ))

In [31]:
frame_QQ_0 = pd.Series(frame_QQ['boat'][0:417])
frame_QQ_1 = pd.Series(frame_QQ['survived'][0:417])
data_QQ = {'boat':frame_QQ_0,'survived':frame_QQ_1}
frame_QQ_boat = pd.DataFrame(data_QQ)

In [32]:
frame_QQ_boat = frame_QQ_boat.sort_values(by = 'boat',ascending=True)
frame_QQ_boat.index = range(len(frame_QQ_boat))

In [33]:
bboat = ['1','2','3','4','5','6','7','8','9','10','11','12','13','14','15','16','A','B','C','D']
for i in range(len(bboat)):
    q = 0
    r = 0
    s = len(frame_QQ_boat.loc[frame_QQ_boat['boat'] == bboat[i]])
    q = frame_QQ_boat.loc[frame_QQ_boat['boat'] == bboat[i]].sum()    
    print( 'boat : ',bboat[i] ,'Survived_rate = ',q[1]/s*100 ,'%' )

boat :  1 Survived_rate =  100.0 %
boat :  2 Survived_rate =  100.0 %
boat :  3 Survived_rate =  100.0 %
boat :  4 Survived_rate =  100.0 %
boat :  5 Survived_rate =  100.0 %
boat :  6 Survived_rate =  100.0 %
boat :  7 Survived_rate =  100.0 %
boat :  8 Survived_rate =  100.0 %
boat :  9 Survived_rate =  100.0 %
boat :  10 Survived_rate =  100.0 %
boat :  11 Survived_rate =  100.0 %
boat :  12 Survived_rate =  94.73684210526315 %
boat :  13 Survived_rate =  100.0 %
boat :  14 Survived_rate =  96.55172413793103 %
boat :  15 Survived_rate =  100.0 %
boat :  16 Survived_rate =  100.0 %
boat :  A Survived_rate =  60.0 %
boat :  B Survived_rate =  83.33333333333334 %
boat :  C Survived_rate =  100.0 %
boat :  D Survived_rate =  93.75 %


In [34]:
##### IF no boat survived rate #####

In [35]:
frame_QQ_0 = pd.Series(frame_QQ['boat'][417:1000])
frame_QQ_1 = pd.Series(frame_QQ['survived'][417:1000])
data_QQ = {'boat':frame_QQ_0,'survived':frame_QQ_1}
frame_QQ_no_boat = pd.DataFrame(data_QQ)

In [36]:
frame_QQ_no_boat.sum(),frame_QQ_no_boat.isnull().sum()
print('no_boat_survived_rate : ',14/583*100 ,' % ')

no_boat_survived_rate :  2.401372212692967  % 


 - # 可以看出有無搭上船以及哪艘船對存活率的判斷很重要

In [70]:
###  ↓ ↓ ↓ ↓ ↓ ↓ 將文字替換成數字用以訓練 

 - Convert data_boat_survived_rate  to [0 ~ 100 % ]  ， 由於有無搭上船以及搭哪艘船對生存率影響很重要 ，所以有對各種船的生存率去做調整

In [37]:
h = {'1':100,'2':100,'3':100,'4':100,'5':100,'6':100,'7':100,'8':100,'9':100,'10':100,'11':100,'12':94.73,'13':100,'14':96.55,'15':100,'16':100,'A':60,'B':83.33,'C':100,'D':93.75,'NaN':2.4}

In [38]:
x = data_train['boat']
y = data_test['boat']
boat_data_train = x.map(h)
boat_data_test = y.map(h)
boat_data_train = boat_data_train.fillna(2.4)
boat_data_test = boat_data_test.fillna(2.4)

 - Convert data_sex [ female  &  male ] to [0 & 1]

In [39]:
z = {'female': 0, 'male': 1}

In [40]:
data_test.keys()

Index(['pclass', 'survived', 'name', 'sex', 'age', 'sibsp', 'parch', 'ticket',
       'fare', 'cabin', 'embarked', 'boat', 'body', 'home.dest'],
      dtype='object')

In [41]:
x = data_train['sex']
y = data_test['sex']
sex_data_train = x.map(z)
sex_data_test = y.map(z)

 - Convert data_embarked [ S  &  C  &  Q ] to [ 0 & 1 & 2 ]

In [42]:
g = {'S': 0, 'C': 1, 'Q': 2}

In [43]:
x = data_train['embarked']
y = data_test['embarked']
embarked_data_train = x.map(g)
embarked_data_test = y.map(g)

 - Get data  [  survived  ,  sex  ,  age  ,  sibsp  ,  ticket  ] to train

In [44]:
data_train_tr = data_train.iloc[:,[4, 5 ,6 ,8 ,1]].values
data_test_te = data_test.iloc[:,[4, 5 ,6 ,8]].values

 - Use data [  survived  ,   sex  ,   age  ,   sibsp  ,   ticket  ,   embarked   ] to  train

In [45]:
# train = all ， survived = 14.5%

In [46]:
frame_train = pd.DataFrame(data_train_tr)
frame_0 = pd.Series(frame_train[0][0:1000])
frame_1 = pd.Series(frame_train[1][0:1000])
frame_2 = pd.Series(frame_train[2][0:1000])
frame_3 = pd.Series(frame_train[3][0:1000])
frame_4 = pd.Series(frame_train[4][0:1000])
frame_5 = pd.Series(sex_data_train[0:1000])
frame_6 = pd.Series(embarked_data_train[0:1000])
frame_7 = pd.Series(boat_data_train[0:1000])
data_train = {'age':frame_0,'sibsp':frame_1,'parch':frame_2,'fare':frame_3,'survived':frame_4,'sex':frame_5,'embarked':frame_6,'boat':frame_7}
frame_train = pd.DataFrame(data_train)

In [47]:
frame_test = pd.DataFrame(data_test_te)
frame_0 = pd.Series(frame_test[0])
frame_1 = pd.Series(frame_test[1])
frame_2 = pd.Series(frame_test[2])
frame_3 = pd.Series(frame_test[3])
frame_5 = pd.Series(sex_data_test[0:1000])
frame_6 = pd.Series(embarked_data_test[0:1000])
frame_7 = pd.Series(boat_data_test[0:1000])
data_test = {'age':frame_0,'sibsp':frame_1,'parch':frame_2,'fare':frame_3,'sex':frame_5,'embarked':frame_6,'boat':frame_7}
frame_test = pd.DataFrame(data_test)

 - Test Model accuracy_rate

In [48]:
# train = 0.75(data_train) ；test = 0.25(data_train) ， Model accuracy_rate = 70%

In [49]:
# frame_train = pd.DataFrame(data_train_tr)
# frame_0 = pd.Series(frame_train[0][0:750])
# frame_1 = pd.Series(frame_train[1][0:750])
# frame_2 = pd.Series(frame_train[2][0:750])
# frame_3 = pd.Series(frame_train[3][0:750])
# frame_4 = pd.Series(frame_train[4][0:750])
# frame_5 = pd.Series(sex_data_train[0:750])
# frame_6 = pd.Series(embarked_data_train[0:750])
# frame_7 = pd.Series(boat_data_train[0:750])
# data_train = {'age':frame_0,'sibsp':frame_1,'parch':frame_2,'fare':frame_3,'survived':frame_4,'sex':frame_5,'embarked':frame_6,'boat':frame_7}
# frame_train = pd.DataFrame(data_train)

In [50]:
# boat_data_train
# boat_data_test

In [51]:
# frame_test = pd.DataFrame(data_train_tr)
# frame_0 = pd.Series(frame_test[0][750:1000])
# frame_1 = pd.Series(frame_test[1][750:1000])
# frame_2 = pd.Series(frame_test[2][750:1000])
# frame_3 = pd.Series(frame_test[3][750:1000])
# frame_5 = pd.Series(sex_data_train[750:1000])
# frame_6 = pd.Series(embarked_data_train[750:1000])
# frame_7 = pd.Series(boat_data_train[750:1000])
# data_test = {'age':frame_0,'sibsp':frame_1,'parch':frame_2,'fare':frame_3,'sex':frame_5,'embarked':frame_6,'boat':frame_7}
# frame_test = pd.DataFrame(data_test)

In [52]:
frame_train = frame_train.fillna(frame_train.mean())
frame_test = frame_test.fillna(frame_test.mean())

In [53]:
def get_testing_labels(training_samples, training_labels, testing_samples):
    normalizer       = Normalizer().fit(training_samples)
    training_samples = normalizer.transform(training_samples)
    testing_samples  = normalizer.transform(testing_samples)

    pca              = PCA(n_components=0.8, whiten=True).fit(training_samples)
    training_samples = pca.transform(training_samples)
    testing_samples  = pca.transform(testing_samples)

    svm_classifier   = SVC(C=2.8).fit(training_samples, training_labels)
    return svm_classifier.predict(testing_samples)

In [54]:
def main():
    # Read training and testing data
    training_df = frame_train
    testing_df  = frame_test 

    # Get training samples and labels from training data
    training_labels     = training_df['survived'].values
    training_samples    = training_df.drop(['survived'], axis = 1).values
    # Get testing samples from testing data
    testing_samples     = testing_df.values
    testing_labels      = get_testing_labels(training_samples, training_labels, testing_samples)
    
    # Writing testing labels to CSV
    testing_labels_df   = pd.DataFrame(testing_labels, columns=['Label'])
    testing_labels_df.index     += 1
    testing_labels_df.index.name = 'ImageId'
    testing_labels_df.to_csv('Predict_survived.csv', sep=',')

In [55]:
if __name__ == "__main__":
    main()

In [56]:
###############################################################################
###############################################################################
###############################################################################

 -  - Model predict accuracy rate

In [57]:
###############################################################################
###############################################################################
###############################################################################

In [58]:
# train = (data_train) ；test = (data_train) ， Model accuracy_rate = 97%

In [59]:
frame_train = pd.DataFrame(data_train_tr)
frame_0 = pd.Series(frame_train[0][0:1000])
frame_1 = pd.Series(frame_train[1][0:1000])
frame_2 = pd.Series(frame_train[2][0:1000])
frame_3 = pd.Series(frame_train[3][0:1000])
frame_4 = pd.Series(frame_train[4][0:1000])
frame_5 = pd.Series(sex_data_train[0:1000])
frame_6 = pd.Series(embarked_data_train[0:1000])
frame_7 = pd.Series(boat_data_train[0:1000])
data_train = {'age':frame_0,'sibsp':frame_1,'parch':frame_2,'fare':frame_3,'survived':frame_4,'sex':frame_5,'embarked':frame_6,'boat':frame_7}
frame_train = pd.DataFrame(data_train)

In [60]:
frame_test = pd.DataFrame(data_train_tr)
frame_0 = pd.Series(frame_test[0][0:1000])
frame_1 = pd.Series(frame_test[1][0:1000])
frame_2 = pd.Series(frame_test[2][0:1000])
frame_3 = pd.Series(frame_test[3][0:1000])
frame_5 = pd.Series(sex_data_train[0:1000])
frame_6 = pd.Series(embarked_data_train[0:1000])
frame_7 = pd.Series(boat_data_train[0:1000])
data_test = {'age':frame_0,'sibsp':frame_1,'parch':frame_2,'fare':frame_3,'sex':frame_5,'embarked':frame_6,'boat':frame_7}
frame_test = pd.DataFrame(data_test)

 - 缺失值( nan ) 用平均去帶入
 - 參考網站 : http://www.ppvke.com/Blog/archives/36612

In [61]:
frame_train = frame_train.fillna(frame_train.mean())
frame_test = frame_test.fillna(frame_test.mean())

In [62]:
def get_testing_labels(training_samples, training_labels, testing_samples):
    normalizer       = Normalizer().fit(training_samples)
    training_samples = normalizer.transform(training_samples)
    testing_samples  = normalizer.transform(testing_samples)

    pca              = PCA(n_components=0.8, whiten=True).fit(training_samples)
    training_samples = pca.transform(training_samples)
    testing_samples  = pca.transform(testing_samples)

    svm_classifier   = SVC(C=2.8).fit(training_samples, training_labels)
    return svm_classifier.predict(testing_samples)

In [63]:
def main():
    # Read training and testing data
    training_df = frame_train
    testing_df  = frame_test 

    # Get training samples and labels from training data
    training_labels     = training_df['survived'].values
    training_samples    = training_df.drop(['survived'], axis = 1).values
    # Get testing samples from testing data
    testing_samples     = testing_df.values
    testing_labels      = get_testing_labels(training_samples, training_labels, testing_samples)
    
    # Writing testing labels to CSV
    testing_labels_df   = pd.DataFrame(testing_labels, columns=['Label'])
    testing_labels_df.index     += 1
    testing_labels_df.index.name = 'ImageId'
    testing_labels_df.to_csv('sample_submission_1.csv', sep=',')

In [64]:
if __name__ == "__main__":
    main()

 - Model predict accuracy rate

In [65]:
data_train = pd.read_csv("titanicTrain.csv")
data_predict = pd.read_csv("sample_submission_1.csv")

In [66]:
data_train.keys()

Index(['pclass', 'survived', 'name', 'sex', 'age', 'sibsp', 'parch', 'ticket',
       'fare', 'cabin', 'embarked', 'boat', 'body', 'home.dest'],
      dtype='object')

In [67]:
data_predict.keys()

Index(['ImageId', 'Label'], dtype='object')

In [68]:
frame_train = pd.DataFrame(data_train)
frame_predict = pd.DataFrame(data_predict)
predict_before = pd.Series(frame_train['survived'][0:1000])
predict_after  = pd.Series(frame_predict['Label'][0:1000])
data_predict = {'Before':predict_before,'After':predict_after}
predict_train = pd.DataFrame(data_predict)

In [69]:
g = 0
for i in range(len(predict_train)):
    g = g+np.abs(predict_train['After'][i]-predict_train['Before'][i])
print('Predict_accuracy_rate = ',(1-(g/len(predict_train)))*100,' %')

Predict_accuracy_rate =  96.6  %
